# Reconciliation Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import base64
from config import Config
from util.helpers import (
    convert_utc_date_to_inclusion_dates, get_auth_token
)

import os

from datetime import datetime, timezone
import requests
from IPython import get_ipython
from IPython.display import display, Markdown

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
# This will create the connection to the database and prep the jupyter magic for SQL
%sql $Config.SQLALCHEMY_DATABASE_URI

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill


Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
import sys
%sql set time zone 'UTC';
db_connection_test = get_ipython().run_line_magic("sql", "SELECT 1")
if not db_connection_test:
    sys.exit('Database connection error')

In [ ]:
print(f"Using from_date: {from_date} and to_date: {to_date} UTC")

def save_to_csv(target, report_type):
    date_string, _ = convert_utc_date_to_inclusion_dates(from_date, to_date)
    if report_type == "summary":
        filename_summary = (
        os.path.join(os.getcwd(), r"data/")
            + partner_code
            + "_reconciliation_summary_"
            + date_string.replace(' to ', '_')
            + ".csv"
        )
        with open(filename_summary, "w") as f:
            f.write(f"Reconciliation Summary: {date_string} - Note: this includes more payment statuses than just PAID. This is based off of payment_date and refund_date.\n\n")
            if target is None or target.empty:
                f.write("No Data Retrieved")
            else:
                target.to_csv(f, sep=",", encoding="utf-8", index=False)
    elif report_type == "disbursement":
        if partner_code in partners_for_disbursement_summary:
            filename_disbursed = os.path.join(
                os.getcwd(),
                "data",
                f"{partner_code}_reconciliation_disbursed_"
                + date_string.replace(' to ', '_')
                + ".csv",
            )
            print(f"Saving CSV for partner_code: {partner_code} at {filename_disbursed}")
            with open(filename_disbursed, "w") as f:
                f.write(f"Reconciliation Disbursed: {date_string} - Note: this includes disbursement reversals as well. This is based off of the disbursement_date and disbursement_reversal_date.\n\n")
                if target is None or target.empty:
                    f.write("No Data Retrieved")
                else:
                    target.to_csv(f, sep=",", encoding="utf-8", index=False)

def get_stat_fee_gst_gl():
    query = f"""
            SELECT dc.client
                || '.' || dc.responsibility_centre
                || '.' || dc.service_line
                || '.' || dc.stob
                || '.' || dc.project_code AS distribution_string
            FROM fee_schedules fs
            JOIN distribution_code_links dcl on dcl.fee_schedule_id = fs.fee_schedule_id
            JOIN distribution_codes dc on dc.distribution_code_id = dcl.distribution_code_id
            WHERE fs.corp_type_code = :partner_code
              AND dc.statutory_fees_gst_distribution_code_id is not null
            LIMIT 1
    """
    results = get_ipython().run_cell_magic('sql', '', query)

    if results and len(results) > 0:
        return results[0][0]
    return None

Summary Query

In [ ]:
%%sql reconciliation_summary  <<
SELECT
    i.id as transaction_id,
    (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date_pacific,
    created_name,
    pa.auth_account_id as account_number,
    pa.name as account_name,
    total - service_fees - coalesce(gst, 0.00) as stat_fee,
    pli_summary.statutory_fees_gst as stat_fee_gst,
    service_fees,
    pli_summary.service_fees_gst as service_fee_gst,
    total - refund as subtotal,
    coalesce(gst, 0.00) as gst_total,
    total,
    refund,
    payment_method_code,
    corp_type_code,
    payment_date,
    refund_date,
    invoice_status_code,
    folio_number,
    (select string_agg(quantity || 'x - ' || filing_type_code || ' - ' || description || ' - $' || pli.total, ',') from payment_line_items pli JOIN fee_schedules fs on fs.fee_schedule_id = pli.fee_schedule_id where invoice_id = i.id) as payment_line_items
FROM
    invoices i
LEFT JOIN
    payment_accounts pa ON i.payment_account_id = pa.id
LEFT JOIN (
        SELECT
            pli.invoice_id,
            SUM(pli.statutory_fees_gst) statutory_fees_gst,
            SUM(pli.service_fees_gst) service_fees_gst
        FROM payment_line_items pli
        GROUP BY pli.invoice_id
    ) pli_summary on pli_summary.invoice_id = i.id
WHERE
    corp_type_code = :partner_code
    AND total > 0
    AND invoice_status_code in ('PAID', 'CREDITED', 'REFUNDED', 'REFUND_REQUESTED')
    AND payment_method_code in ('PAD', 'EJV', 'EFT', 'DIRECT_PAY', 'ONLINE_BANKING')
    AND ((payment_date >= :from_date and payment_date <= :to_date)
         OR (refund_date >= :from_date and refund_date <= :to_date))
ORDER BY
    CASE
        WHEN refund_date IS NOT NULL AND invoice_status_code = 'PAID' THEN 1
        ELSE 0
    END,
    i.id;

Disbursement

In [ ]:
save_to_csv(reconciliation_summary.DataFrame(), "summary")
# Define the partner codes for which the disbursement summary should be executed
partners_for_disbursement_summary = Config.PARTNER_CODES_DISBURSEMENT.split(",")

print(f"Processing for partner_code: {partner_code}")
print(f"Partners for disbursement summary: {partners_for_disbursement_summary}")

# Only date part, no time saved for this field, also I believe this is pacific, it comes straight
# from the feedback files.
from_date_disbursement = datetime.strptime(from_date.split(' ')[0], "%Y-%m-%d").date()
to_date_disbursement = datetime.strptime(to_date.split(' ')[0], "%Y-%m-%d").date()
print(f'Using from_date_disbursement: {from_date_disbursement} and to_date_disbursement: {to_date}')
if partner_code in partners_for_disbursement_summary:
    print(f"Partner code {partner_code} found in the list, executing SQL query.")
    query = f"""
    SELECT
        i.id as transaction_id,
        (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date_pacific,
        created_name,
        pa.auth_account_id as account_number,
        pa.name as account_name,
        total - service_fees - coalesce(gst, 0.00) as stat_fee,
        pli_summary.statutory_fees_gst as stat_fee_gst,
        service_fees,
        pli_summary.service_fees_gst as service_fee_gst,
        total - refund as subtotal,
        coalesce(gst, 0.00) as gst_total,
        total,
        refund,
        total - service_fees - coalesce(gst, 0.00) + pli_summary.statutory_fees_gst as disbursed_amount,
        CASE
            WHEN refund - service_fees - pli_summary.service_fees_gst < 0 THEN 0
            ELSE refund - service_fees - pli_summary.service_fees_gst
        END AS reversed_amount,
        payment_method_code,
        corp_type_code,
        payment_date,
        refund_date,
        invoice_status_code,
        folio_number,
        disbursement_date::date as disbursement_date_pacific,
        disbursement_reversal_date::date disbursement_reversal_date_pacific,
        disbursement_status_code,
        feedback_on::date as partial_refund_disbursement_reversal_date_pacific,
        (select string_agg(quantity || 'x - ' || filing_type_code || ' - ' || description || ' - $' || pli.total , ',') from payment_line_items pli join fee_schedules fs on fs.fee_schedule_id = pli.fee_schedule_id where invoice_id = i.id) as payment_line_items
    FROM
        invoices i
    LEFT JOIN
        payment_accounts pa ON i.payment_account_id = pa.id
    LEFT JOIN (
        SELECT
            rp.invoice_id,
            SUM(rp.refund_amount) AS total_refund,
            max(pd.feedback_on) as feedback_on
        FROM refunds_partial rp
        JOIN partner_disbursements pd
            ON pd.target_id = rp.id
            AND pd.target_type = 'partial_refund'
        WHERE pd.feedback_on BETWEEN :from_date_disbursement AND :to_date_disbursement
        GROUP BY rp.invoice_id
    ) rp ON rp.invoice_id = i.id
    LEFT JOIN (
        SELECT
            pli.invoice_id,
            SUM(pli.statutory_fees_gst) statutory_fees_gst,
            SUM(pli.service_fees_gst) service_fees_gst
        FROM payment_line_items pli
        GROUP BY pli.invoice_id
    ) pli_summary on pli_summary.invoice_id = i.id
    WHERE corp_type_code = :partner_code
    AND invoice_status_code in ('PAID', 'CREDITED', 'REFUNDED')
    AND payment_method_code in ('PAD','EJV', 'EFT', 'DIRECT_PAY', 'ONLINE_BANKING')
    AND ((disbursement_date >= :from_date_disbursement AND disbursement_date <= :to_date_disbursement)
        OR (disbursement_reversal_date >= :from_date_disbursement AND disbursement_reversal_date <= :to_date_disbursement)
        OR (feedback_on >= :from_date_disbursement AND feedback_on <= :to_date_disbursement))
    ORDER BY feedback_on ASC NULLS FIRST, i.id;
    """

    display(Markdown(f"## Running query for partner: {partner_code}"))
    results = get_ipython().run_cell_magic('sql', '', query)
    reconciliation_disbursed = results.DataFrame()  # Convert the results to a DataFrame for later use
else:
    print(f"Partner code {partner_code} not in the list, skipping SQL query.")
    reconciliation_disbursed = None
save_to_csv(reconciliation_disbursed, "disbursement")

In [ ]:

partner_details = {
    "CSO": {
        "companyName": "Ministry of Justice",
        "addressLine1": "PO Box 9249, Stn Prov Govt",
        "addressLine2": "6th Floor, 850 Burdett Avenue",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9J2"
    },
    "VS": {
        "companyName": "Vital Statistics Agency",
        "addressLine1": "PO Box 9657, Stn Prov Govt",
        "addressLine2": "",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9P3"
    },
    "RPT": {
        "companyName": "Property Taxation Branch",
        "addressLine1": "Ministry of Provincial Revenue",
        "addressLine2": "4th Floor, 1802 Douglas Street",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8T 4K6"
    },
    "ESRA": {
        "companyName": "Site Remediation Program, Authorizations and Remediation Branch",
        "addressLine1": "Ministry of Environment and Parks",
        "addressLine2": "525 Superior Street, 3rd floor",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8V 0C5"
    },
    "STRR": {
        "companyName": "Ministry of Housing and Municipal Affairs",
        "addressLine1": "PO BOX 9844, STN PROV GOVT",
        "addressLine2": "4th Floor, 614 Humboldt Street",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 1A4"
    }
}

def generate_report(partner_code):
    if not Config.REPORT_API_URL:
        raise ValueError("The REPORT_API_URL environment variable is not set or is empty")

    url = Config.REPORT_API_URL + '/reports'
    headers = {
        'Authorization': f'Bearer {get_auth_token()}',
        'Content-Type': 'application/json',
        'Accept': 'application/pdf'
    }

    table_rows = [
        {
            'registry': partner_code,
            'transactionCounts': 0,
            'totalPayment': 0,
            'totalDisbursement': 0,
            'totalStatFeeGst': 0
        }
    ]
    disbursed_gst_amount = 0
    if reconciliation_summary:
        csv_paid_only = reconciliation_summary.DataFrame() \
            .query("invoice_status_code == 'PAID' and (payment_date >= @from_date and payment_date <= @to_date)").get('total', 0)
        csv_paid_total = csv_paid_only.sum()
        csv_count_total = csv_paid_only.count()
        disbursed_total_amount = 0
        if reconciliation_disbursed is not None and reconciliation_disbursed.empty is False:
            disbursed_query = reconciliation_disbursed.query("disbursement_status_code == 'COMPLETED' and (disbursement_date_pacific >= @from_date_disbursement and disbursement_date_pacific <= @to_date_disbursement)")
            disbursed_total_amount = disbursed_query.get('disbursed_amount', 0).sum()
            disbursed_gst_amount = disbursed_query.get('stat_fee_gst', 0).sum()
        format_currency = lambda x: f"$ {x:,.2f}"
        format_number = lambda x: f"{x:,.0f}"
        table_rows = [{
            'registry': partner_code,
            'totalPayment': format_currency(csv_paid_total),
            'transactionCounts': format_number(csv_count_total),
            'totalDisbursement': format_currency(disbursed_total_amount),
            'totalDisbursementGst': format_currency(disbursed_gst_amount),
        }]

    current_date = datetime.now(tz=timezone.utc).strftime("%B %d, %Y")

    details = partner_details.get(partner_code, {})
    if not details:
        raise ValueError(f"No details found for partner code: {partner_code}")

    # Define the request body
    date_range, _ = convert_utc_date_to_inclusion_dates(from_date, to_date, 'full')
    data = {
        "templateVars": {
            "date": current_date,
            "companyName": details["companyName"],
            "addressLine1": details["addressLine1"],
            "addressLine2": details["addressLine2"],
            "city": details["city"],
            "province": details["province"],
            "areaCode": details["areaCode"],
            "firstName": partner_code,
            "dateRange": date_range,
            "tableRows": table_rows,
            "hasGst": disbursed_gst_amount > 0,
            "gstGL": get_stat_fee_gst_gl()
        },
        "templateName": "revenue_letter",
        "reportName": "revenue_letter"
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        pdf_content = response.content
        date_string, _ = convert_utc_date_to_inclusion_dates(from_date, to_date)
        date_string = date_string.replace(' to ', '_')
        pdf_filename = os.path.join(os.getcwd(), 'data', f'{partner_code}_revenue_letter_{date_string}.pdf')
        with open(pdf_filename, 'wb') as pdf_file:
            pdf_file.write(pdf_content)

        print("PDF report saved successfully as 'payment_receipt.pdf'")
    else:
        print('Failed to get the report:', response.text)
generate_report(partner_code)